In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Colab Notebooks/22-1 추천시스템 스터디

/content/gdrive/MyDrive/Colab Notebooks/22-1 추천시스템 스터디


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

In [4]:
df = pd.read_csv('tmdb_5000_movies.csv', encoding = 'utf-8')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
df.shape

(4803, 20)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

## 데이터 전처리

### 사용할 컬럼 선택

In [7]:
data = df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]
data.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [8]:
data.shape

(4803, 8)

### dict 형태 전처리

In [11]:
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
data['genres'] = data['genres'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,6.9,4500,139.082615,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,Action Adventure Crime,6.3,4466,107.376788,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller,7.6,9106,112.312950,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...
4,49529,John Carter,Action Adventure Science Fiction,6.1,2124,43.926995,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca..."


### 평점 전처리

In [13]:
percent = 0.8
m = data['vote_count'].quantile(percent)
C = data['vote_average'].mean()

def weighted_score(record):
  v = record['vote_count']
  R = record['vote_average']

  return ((v/(v+m) * R) + ((m/(m+v)) * C))

In [14]:
data['weighted_score'] = data.apply(weighted_score, axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
data.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,weighted_score
0,19995,Avatar,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.116845
1,285,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,6.9,4500,139.082615,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.758257
2,206647,Spectre,Action Adventure Crime,6.3,4466,107.376788,spy based on novel secret agent sequel mi6 bri...,A cryptic message from Bond’s past sends him o...,6.263305
3,49026,The Dark Knight Rises,Action Crime Drama Thriller,7.6,9106,112.312950,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.456523
4,49529,John Carter,Action Adventure Science Fiction,6.1,2124,43.926995,based on novel mars medallion space travel pri...,"John Carter is a war-weary, former military ca...",6.097567


In [39]:
data['weighted_score'].sort_values(ascending = False)

1881    8.248353
662     8.096134
3337    8.077404
3232    8.074738
65      8.044250
          ...   
303     5.226248
3746    5.063681
1652    5.053299
210     4.962731
242     4.900811
Name: weighted_score, Length: 4803, dtype: float64

## 콘텐츠 기반 필터링

### tf-idf 벡터화

In [24]:
tfidf_vector = TfidfVectorizer(ngram_range=(1,3))
tfidf_matrix = tfidf_vector.fit_transform(data['genres'] + " " + data['keywords']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
tfidf_matrix.shape

(4803, 92540)

In [26]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = data.title)
print(tfidf_matrix.shape)
tfidf_matrix.head()

(4803, 92540)


,11,11 2001,11 2001 investigative,11 2001 woman,11 helicopter,11 helicopter crash,15th,15th century,15th century ottoman,16th,...,γη,γη mission,γη mission control,卧底肥妈,卧底肥妈 big爆任务,卧底肥妈 big爆任务 超级妈妈,绝地奶霸,绝地奶霸 卧底肥妈,绝地奶霸 卧底肥妈 big爆任务,超级妈妈
title,,,,,,,,,,,,,,,,,,,,,
Avatar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pirates of the Caribbean: At World's End,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Spectre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Dark Knight Rises,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
John Carter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 유사도 구하기

In [28]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [29]:
cosine_sim.shape

(4803, 4803)

In [30]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = data.title, columns = data.title)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(4803, 4803)


title,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,...,On The Downlow,Sanctuary: Quite a Conundrum,Bang,Primer,Cavite,El Mariachi,Newlyweds,"Signed, Sealed, Delivered",Shanghai Calling,My Date with Drew
title,,,,,,,,,,,,,,,,,,,,,
Avatar,1.000000,0.012242,0.007878,0.000972,0.070200,0.010956,0.006028,0.020870,0.012077,0.023220,...,0.003225,0.000000,0.000000,0.013457,0.000000,0.001648,0.017286,0.005918,0.0,0.0
Pirates of the Caribbean: At World's End,0.012242,1.000000,0.004413,0.007597,0.002850,0.069307,0.002665,0.004016,0.017534,0.013063,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001923,0.000000,0.004624,0.0,0.0
Spectre,0.007878,0.004413,1.000000,0.016975,0.034193,0.018033,0.010703,0.036705,0.003635,0.023473,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.028342,0.000000,0.000000,0.0,0.0
The Dark Knight Rises,0.000972,0.007597,0.016975,1.000000,0.001063,0.011662,0.006049,0.015260,0.000000,0.054090,...,0.001274,0.005426,0.002371,0.024887,0.006538,0.014558,0.000000,0.001698,0.0,0.0
John Carter,0.070200,0.002850,0.034193,0.001063,1.000000,0.006284,0.014779,0.059855,0.002348,0.015161,...,0.003529,0.000000,0.000000,0.014725,0.000000,0.001803,0.000000,0.000000,0.0,0.0


In [51]:
def genre_recommendations(target_title, matrix, items, k=10):
    recom_idx_ = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:2*k+1]
    recom_idx = items.iloc[recom_idx_].sort_values('weighted_score', ascending = False)[:k].index
    recom_title = items.iloc[recom_idx, :].title.values
    recom_genre = items.iloc[recom_idx, :].genres.values
    recom_score = items.iloc[recom_idx, :].weighted_score.values
    target_title_list = np.full(len(range(k)), target_title)
    target_genre_list = np.full(len(range(k)), items[items.title == target_title].genres.values)

    d = {
        'target_title':target_title_list,
        'target_genre':target_genre_list,
        'recom_title' : recom_title,
        'recom_genre' : recom_genre,
        'recom_score' : recom_score
    }
    return pd.DataFrame(d)

In [52]:
genre_recommendations('The Dark Knight Rises', cosine_sim_df, data)

,target_title,target_genre,recom_title,recom_genre,recom_score
0,The Dark Knight Rises,Action Crime Drama Thriller,The Dark Knight,Drama Action Crime Thriller,8.044250
1,The Dark Knight Rises,Action Crime Drama Thriller,Batman Begins,Action Crime Drama,7.337898
2,The Dark Knight Rises,Action Crime Drama Thriller,Batman,Fantasy Action,6.715308
3,The Dark Knight Rises,Action Crime Drama Thriller,Superman,Action Adventure Fantasy Science Fiction,6.509226
4,The Dark Knight Rises,Action Crime Drama Thriller,Batman Returns,Action Fantasy,6.415139
5,The Dark Knight Rises,Action Crime Drama Thriller,Panic Room,Crime Drama Thriller,6.324446
6,The Dark Knight Rises,Action Crime Drama Thriller,Zulu,Crime Drama Thriller,6.184723
7,The Dark Knight Rises,Action Crime Drama Thriller,Street Kings,Action Crime Drama Thriller,6.149298
8,The Dark Knight Rises,Action Crime Drama Thriller,Ajami,Crime Drama,6.111582
9,The Dark Knight Rises,Action Crime Drama Thriller,Deadline - U.S.A.,Crime Drama,6.106263
